In [3]:
import os
import glob

import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Polygon

import ee

from zipfile import ZipFile
from zipfile import BadZipfile

import requests

#ee.Authenticate()
ee.Initialize()

In [31]:
## take the ads file
gdf = gpd.read_file('./input/sanjuan_NF_boundary.shp')
#gdf.head()
#gdf.head()
#gdf = gdf[['id', 'SURVEY_YEA', 'geometry']]
gdf['id'] = 1
gdf['SURVEY_YEA'] = 2023
gdf = gdf[['id', 'SURVEY_YEA', 'geometry']]
gdf.head()

,id,SURVEY_YEA,geometry
0,1,2023,"POLYGON ((-108.76696 37.89542, -108.76238 37.8..."


In [3]:
# filter if the API breaks
#gdf = gdf[499:]
#gdf.head()

In [6]:
for index, row in gdf.iterrows():
    print(row['id'])

    end = row['SURVEY_YEA']
    start = end - 11

    start_date = str(start)+'-01-01'
    end_date = str(end)+'-12-31'

    print(f'start is {start_date} end is {end_date}')

    corners = list(row['geometry'].exterior.coords)
    new_polygon = Polygon(corners)

    aoi  = ee.Geometry.Polygon([list(coord) for coord in corners])

    pre_collection = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterBounds(aoi).filterDate(start_date, end_date).select('def')

    #summer_collection = pre_collection.filter(ee.Filter.calendarRange(6, 8, 'month'))
    summer_collection = pre_collection


    image_list = summer_collection.toList(summer_collection.size())

    # create folder for images
    ads_id = row['id']
    folder_path  = './cwd/images/'+str(ads_id)
    print(folder_path)
    # create folders
    try:
        os.makedirs(folder_path)
    except:
        print('Folder in place already')
    # number of images in collection for aoi
    num_images = summer_collection.size().getInfo()
    for item in range(0, num_images):
        # take images
        filter_image = ee.Image(image_list.get(item))
        # clip to aoi
        pre_image = filter_image.clip(aoi)#.unmask()
        # image name
        im_name = str(pre_image.get('system:index').getInfo())
        print(im_name)
        # take url
        try:
            url = pre_image.getDownloadURL({'scale':4000,
                                                    'filePerBand':False, 
                                                    'region':aoi,
                                                    'crs':'EPSG:4326', 
                                                    'maxPixels': 1e13})
            #print(url)
            r = requests.get(url)
            with open(folder_path+'/'+im_name+'.zip', 'wb') as f:
                f.write(r.content)
        except:
            print('no images avialable')
            continue
        # extract the zipfile here
        zip_file = glob.glob('./cwd/images/'+str(ads_id)+'/*zip')
        try:
            zf = ZipFile(zip_file[0])
            zf.extractall('./cwd/images/'+str(ads_id))
            os.remove(zip_file[0])
        except BadZipfile:
            os.remove(zip_file[0])

    print('Down load succefull')

1
start is 2012-01-01 end is 2023-12-31
./cwd/images/1
Folder in place already
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611
201612
201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
2022

In [38]:
import rasterio as ras
from rasterstats import zonal_stats
from rasterio.enums import Resampling as resa
import re
import numpy as np

In [39]:
# list dir images
image_list = glob.glob('./vpd/images/1/*tif')
# Extract the date part (YYYYMM format) and sort by it
sorted_filenames = sorted(image_list, key=lambda x: re.findall(r'\d{6}', x)[0])


for i in range (2000, 2024):
       print(i)
        # empty list to collect bands
       band_list = []
       for path in sorted_filenames:
              if str(i) == path.split('/')[4][:4]:
                     print(path)
                     # read each file
                     src = ras.open(path)
                     band = src.read(1)
                     band_scaled = band * 0.01
                     band_scaled = band_scaled.astype(np.float32)
                     #print(band)
                     band_list.append(band_scaled)
                     # data stack
                     combined_data = np.dstack(band_list)
                     combined_data = combined_data.astype(np.float32)
                     # take mean image
                     #mean_data = np.nanmean(combined_data, axis=2)

                     sum_data = np.sum(combined_data, axis=2)


                     # now zonal stats
                     ds_profile = src.profile
                     ds_affine = ds_profile['transform']
                     #stats = zonal_stats(gdf, mean_data, affine=ds_affine, nodata=0, stats="mean")
                     #zonal_mean = stats[0]['mean']
                     # saving path 
                     save_path = './vpd/composite/'+str(i)+'.tif'

                                   #save images
                     with ras.Env():
                            # image profile
                            profile = src.profile
                            # update profile
                            profile.update(
                                   dtype= ras.float32,
                                   count =1,
                                   compress = 'lzw',
                                   nodata=0
                                   )
                     with ras.open(save_path, 'w', **profile) as dst:
                            dst.write(sum_data.astype(ras.float32), 1)
                            dst.close()
                     print(f'Processing completed for {i}')

                     src.close()
              else:
                     continue
       #print(zonal_mean)

2000
./vpd/images/1/200006.tif
Processing completed for 2000
./vpd/images/1/200007.tif
Processing completed for 2000
./vpd/images/1/200008.tif
Processing completed for 2000
2001
./vpd/images/1/200106.tif
Processing completed for 2001
./vpd/images/1/200107.tif
Processing completed for 2001
./vpd/images/1/200108.tif
Processing completed for 2001
2002
./vpd/images/1/200206.tif
Processing completed for 2002
./vpd/images/1/200207.tif
Processing completed for 2002
./vpd/images/1/200208.tif
Processing completed for 2002
2003
./vpd/images/1/200306.tif
Processing completed for 2003
./vpd/images/1/200307.tif
Processing completed for 2003
./vpd/images/1/200308.tif
Processing completed for 2003
2004
./vpd/images/1/200406.tif
Processing completed for 2004
./vpd/images/1/200407.tif
Processing completed for 2004
./vpd/images/1/200408.tif
Processing completed for 2004
2005
./vpd/images/1/200506.tif
Processing completed for 2005
./vpd/images/1/200507.tif
Processing completed for 2005
./vpd/images/1/200

In [40]:
gdf = gpd.read_file('./input/sanjuan_NF_boundary.shp')
gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPEAREA,SHAPELEN,geometry
0,194196,295494010328,San Juan National Forest,2093914.595,0.863366,9.020643,"POLYGON ((-108.76696 37.89542, -108.76238 37.8..."


In [41]:
# zonal stats
# list images in composited
comp_list = glob.glob('./vpd/composite/*.tif')
sorted_filenames = sorted(comp_list, key=lambda x: re.findall(r'\d{4}', x)[0])

year_list = []
clim_list = []
for file in sorted_filenames:
    year = file.split('/')[3][:4]
    src = ras.open(file)
    band = src.read(1)

    ds_profile = src.profile
    ds_affine = ds_profile['transform']
    stats = zonal_stats(gdf, band, affine=ds_affine, nodata=0, stats="mean")
    zonal_mean = stats[0]['mean']
    year_list.append(year)
    clim_list.append(zonal_mean)
    print(year)
    print(zonal_mean)

2000
2.939192927293535
2001
2.8111960919983185
2002
2.8678027781670403
2003
2.851659207422459
2004
2.6756053519712255
2005
2.6866068248435164
2006
2.7407025848981688
2007
2.937414058120796
2008
2.755455940828195
2009
2.4431988041853514
2010
2.5500747384155455
2011
2.6805830472253365
2012
2.931225695417601
2013
2.4765769148799515
2014
2.4097157750607248
2015
2.4309415945557737
2016
3.1567412007193574
2017
3.1954708812126307
2018
3.50146484375
2019
3.6051421343656576
2020
3.6284303764947685
2021
3.3936773052130045
2022
2.6322121841134156
2023
3.635007576022982


In [42]:
clim_df = pd.DataFrame()
clim_df['year'] = year_list
clim_df['vpd'] = clim_list
clim_df

,year,vpd
0,2000,2.939193
1,2001,2.811196
2,2002,2.867803
3,2003,2.851659
4,2004,2.675605
5,2005,2.686607
6,2006,2.740703
7,2007,2.937414
8,2008,2.755456
9,2009,2.443199


In [43]:
clim_df.to_csv('./vpd.csv')

In [44]:
# open dfs
combined_df = pd.read_csv('./SJ.csv')
combined_df

,year,vpd,pdsi,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,2000,2.939193,-7.540778,NaN,NaN,NaN,NaN
1,2001,2.811196,2.001001,NaN,NaN,NaN,NaN
2,2002,2.867803,-15.571839,NaN,NaN,NaN,NaN
3,2003,2.851659,-8.271898,NaN,NaN,NaN,NaN
4,2004,2.675605,-5.071943,NaN,NaN,NaN,NaN
5,2005,2.686607,2.706338,NaN,NaN,NaN,NaN
6,2006,2.740703,-7.021046,NaN,NaN,NaN,NaN
7,2007,2.937414,3.383348,NaN,NaN,NaN,NaN
8,2008,2.755456,2.204634,NaN,NaN,NaN,NaN
9,2009,2.443199,0.525097,NaN,NaN,NaN,NaN
